<a href="https://colab.research.google.com/github/RizkiRahmann/Prediction-Spam-Email/blob/main/Prediksi_email_spam_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d2342e2ad8dfed00d99c02f403722481b3c45f7d31e5d81a5479dbcef0da0840
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# findspark yang berfungsi sebagai untuk menentukan lokasi pyspark yang sudah terinstall
# mengimport library
import findspark
findspark.init()
findspark.find()
import pyspark

In [ ]:
from pyspark.sql import SparkSession
appName = " FP - BDPL"
spark = SparkSession.builder\
        .master("local")\
        .appName(appName)\
        .config('spark.some.config.option', 'some_value')\
        .getOrCreate()
sc = spark.sparkContext

In [ ]:
# mengimport dataset
df = spark.read.csv('/content/emails.csv',header=True,sep=",",inferSchema=True)

In [ ]:
# inisialisasi tipe data dan menampilkan data
df.printSchema()
df.show()

root
 |-- text: string (nullable = true)
 |-- spam: string (nullable = true)

+--------------------+--------------------+
|                text|                spam|
+--------------------+--------------------+
|Subject: naturall...|                   1|
|Subject: the stoc...|                   1|
|Subject: unbeliev...|                   1|
|Subject: 4 color ...|                   1|
|Subject: do not h...|                   1|
|Subject: great nn...|                   1|
|"Subject: here ' ...|         sea  ports |
|Subject: save you...|                   1|
|Subject: undelive...|                   1|
|Subject: save you...|                   1|
|Subject: las vega...|                   1|
|Subject: save you...|                   1|
|Subject: brighten...|                   1|
|"Subject: wall st...|  yet sti | | pro...|
|Subject: fpa noti...|                   1|
|"Subject: search ...|                   1|
|Subject: only our...|                   1|
|Subject: localize...|                   1

In [ ]:
# seleksi fitur spam dan menampilkan sebanyak 5 baris
df.select('spam').show(5,truncate=False)

+----+
|spam|
+----+
|1   |
|1   |
|1   |
|1   |
|1   |
+----+
only showing top 5 rows



In [ ]:
# memfilter data spam = 1 atau spam = 0 lalu count data untuk menghitung data
df2 = df.filter((df['spam']=="1")|(df['spam']=="0"))
df2.count()

4194

In [ ]:
# lalu menampilkan jumlah data spam 1 dan 0 dengan groupby
df2.groupBy('spam').count().show()

+----+-----+
|spam|count|
+----+-----+
|   0| 3006|
|   1| 1188|
+----+-----+



In [ ]:
# lalu seleksi fitur text dan mengembalikan nilai kolom pertama
txt1 = df2.select('text').first()

In [ ]:
# menampilkan type data
type(txt1)

pyspark.sql.types.Row

In [ ]:
# import tokenizer, regextokenizer lalu memisahkan kalimat jadi kata lalu menampilkan data
from pyspark.ml.feature import Tokenizer, RegexTokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
tokenized = tokenizer.transform(df2)
tokenized.show(2)

+--------------------+----+--------------------+
|                text|spam|               words|
+--------------------+----+--------------------+
|Subject: naturall...|   1|[subject:, natura...|
|Subject: the stoc...|   1|[subject:, the, s...|
+--------------------+----+--------------------+
only showing top 2 rows



In [ ]:
# pada regex menampilkan kata dengan pattern \\W artinya karakter bukan kata
regexTokenized = regexTokenizer.transform(df2)
regexTokenized.show(2)

+--------------------+----+--------------------+
|                text|spam|               words|
+--------------------+----+--------------------+
|Subject: naturall...|   1|[subject, natural...|
|Subject: the stoc...|   1|[subject, the, st...|
+--------------------+----+--------------------+
only showing top 2 rows



In [ ]:
# lalu import stopwordremover digunakan untuk memisahkan tanda baca
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
data = remover.transform(regexTokenized)
data.show(2)

+--------------------+----+--------------------+--------------------+
|                text|spam|               words|            filtered|
+--------------------+----+--------------------+--------------------+
|Subject: naturall...|   1|[subject, natural...|[subject, natural...|
|Subject: the stoc...|   1|[subject, the, st...|[subject, stock, ...|
+--------------------+----+--------------------+--------------------+
only showing top 2 rows



In [ ]:
# lalu mendesain dan mengola tipe data berdasarkan data kata dan data yang telah difilter
data.select('words').first()
data.select('filtered').first()
def mots(txt):
    return [w for w in txt if ((len(w) > 2) & (w != 'subject'))  ]
str = ["subject","sam","de","fat","*","--"]
print(mots(str))

['sam', 'fat']


In [ ]:
# lalu import udf untuk memanipulasi data
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
mots = udf(mots,ArrayType(StringType()))

toNum = lambda str:int(str)
toNum('3')+2
toNum = udf(toNum,'int')

data2 = data.withColumn('motsPur', mots(data["filtered"]))
data2 = data2.withColumn('label', toNum(data["spam"]))
data2.select('motsPur').first()
data2.select('label').show(5)

+-----+
|label|
+-----+
|    1|
|    1|
|    1|
|    1|
|    1|
+-----+
only showing top 5 rows



In [ ]:
# import vector untuk menghitung jumlah vector
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="motsPur", outputCol="features")
data3 = cv.fit(data2).transform(data2)

In [ ]:
train, test = data3.randomSplit([0.7,0.3],seed=100)

In [ ]:
# import naive bayes dan melakukan modeling data
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
# train the model
model = nb.fit(train)
predictions = model.transform(test)
predictions.show(5)

+--------------------+----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|                text|spam|               words|            filtered|             motsPur|label|            features|       rawPrediction|         probability|prediction|
+--------------------+----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|"Subject: : : fas...|   1|[subject, fast, a...|[subject, fast, a...|[fast, acting, vi...|    1|(27040,[4,15,17,1...|[-452.16724560479...|[7.73161550257495...|       1.0|
|"Subject: best pr...|   1|[subject, best, p...|[subject, best, p...|[best, product, 2...|    1|(27040,[45,57,73,...|[-337.30366065851...|[7.47007172536415...|       1.0|
|"Subject: bio ' s...|   0|[subject, bio, s,...|[subject, bio, hi...|[bio, john, sorry...|    0|(27040,[9,35,59,8...|[-205.33504061591...|[0.9999

In [ ]:
# evaluasi model
test2 = test.select('features')
predictions2 = model.transform(test2)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

accuracy = evaluator.evaluate(predictions,{evaluator.metricName:"accuracy"})
f1 = evaluator.evaluate(predictions,{evaluator.metricName:"f1"})
TP = evaluator.evaluate(predictions,{evaluator.metricName:"truePositiveRateByLabel"})
FP = evaluator.evaluate(predictions,{evaluator.metricName:"falsePositiveRateByLabel"})
precisionByLabel = evaluator.evaluate(predictions,{evaluator.metricName:"precisionByLabel"})
recallByLabel = evaluator.evaluate(predictions,{evaluator.metricName:"recallByLabel"})

print("accuracy : ",accuracy)
print("F1 : ",f1)
print("TP : ",TP)
print("FP : ",FP)
print("precisionByLabel : ",precisionByLabel)
print("recallByLabel : ",recallByLabel)


accuracy :  0.9911290322580645
F1 :  0.991153617818902
TP :  0.9897377423033067
FP :  0.005509641873278237
precisionByLabel :  0.9977011494252873
recallByLabel :  0.9897377423033067
